# Sky Visualization from the LCD dataset

The Local Climatological Data (LCD) summaries provide a synopsis of climatic values for a single weather
station over a specific month. The summaries are a product of surface observations from both manual and
automated (`AWOS`, `ASOS`) stations with source data taken from the National Centers for Environmental
Information’s Integrated Surface Data (`ISD`) dataset. Geographic availability includes thousands of locations
worldwide. Climatic values given include hourly, daily, and monthly measurements of temperature, dew point,
humidity, winds, sky condition, weather type, atmospheric pressure and more.

## Sky Conditions 
A report of each cloud layer (up to 3) giving the following information.

Each layer given in the following format: `ccc`:`ll`-`xxx` where:

`ccc` and `ll` are the coverage of a layer is in oktas (i.e. eighths) of sky covered by cloud as per the following table:

| ccc      | ll | description
|----------|-----|--------
| CLR      |   0 | clear sky
| FEW      | 1-2 | few clouds
| SCT      | 3-4 | scattered clouds
| BKN      | 5-8 | broken clouds
| OVC      | 8   | overcast
| VV       | 9-10| obscuration (full, partial)
 
And `xxx` is the Cloud base height at lowest point of layer. In the case of an obscuration this value represents the vertical visibility from the point of observation. Given in hundreds of feet (e.g. 50 = 5000 ft, 120 = 12000 feet).

In some cases a cloud base height will be given without the corresponding cloud amount. In these case the cloud amount is missing or not reported.

Up to 3 layers can be reported however by definition when clear skies are reported it will be reported as only one layer as `CLR`-`00`.

**Note**: Since up to 3 cloud layers can be reported, the full state of the sky can best be determined by the contraction given for the last layer. In other words if three layers are reported and the third layer uses `BKN` then the total state of sky is `BKN` which is similar in definition to *mostly cloudy.* `OVC` is similar to *cloudy* or overcast and `FEW` or `SCT` is similar to *partly cloudy.* It should also be noted that in cases where there are more than 3 cloud layers, the highest layers will not be reported. 

### Run this cell for additional info

In [1]:
import os
with open('link.txt', 'r') as link_file:
    url = link_file.readline()
os.startfile(url)

# Imports

In [2]:
from PIL import Image, ImageDraw, ImageFont
from random import randint
from ipywidgets import interact
import pandas as pd
import re

# Read Data from Table

In [3]:
data = pd.read_csv('ChicagoAirport.csv')
sky_data = data['HourlySkyConditions']
sky_data

0                   SCT:04 70 OVC:08 90
1                             OVC:08 50
2                             OVC:08 29
3                               VV:09 9
4                              OVC:08 8
5                   BKN:07 10 OVC:08 20
6                   BKN:07 12 OVC:08 20
7                   BKN:07 14 OVC:08 25
8                             OVC:08 27
9                   SCT:04 27 OVC:08 35
10                 BKN:07 35 OVC:08 150
11        SCT:04 45 BKN:07 65 OVC:08 80
12                 BKN:07 45 BKN:07 100
13       BKN:07 55 BKN:07 75 OVC:08 150
14       FEW:02 25 BKN:07 60 OVC:08 150
15                  BKN:07 18 OVC:08 50
16        SCT:04 12 BKN:07 30 OVC:08 60
17                             VV:09 10
18                             VV:09 11
19                            OVC:08 17
20                            OVC:08 25
21                             VV:09 15
22                  BKN:07 20 OVC:08 31
23                            OVC:08 34
24                            OVC:08 15


# Parsing Sky Condition Code using External File

In [4]:
def process_sky_data(entry):
    entry2 = f'"{entry}"'
    a = !process_sky_data.py $entry2
    return eval(a[0])

In [5]:
test_data = "FEW:02 55 OVC:08 80"
process_sky_data(test_data)

[(2, 55), (8, 80)]

# Cloud Layer Visualization

In [7]:
@interact
def show_sky(hour=(0, len(data)-1)):
    
    layers = process_sky_data(sky_data[hour])
    
    test_image = Image.new('RGBA', (100, 350), color=(0, 128, 255, 255))
    draw = ImageDraw.Draw(test_image)
    unicode_font = ImageFont.truetype("ARLRDBD.TTF", size=50)

    def draw_layer(coverage, height):
        color = (255, 255, 255, 100) if coverage in range(0, 9) else (0, 0, 0, 255)
        shape = "•" if coverage in range(0, 9) else '_'

        new_base = Image.new('RGBA', (100, 350), color=(0, 0, 0, 0))
        height2 = 350-height
        for _ in range(2*coverage**2):
            new_layer = Image.new('RGBA', (100, 350), color=(0, 0, 0, 0))
            layer_draw = ImageDraw.Draw(new_layer)
            x = randint(-30, 130)
            y = randint(height2-60, height2-30)
            layer_draw.text((x, y), shape, font=unicode_font, fill=color)
            new_base = Image.alpha_composite(new_base, new_layer)
        return new_base
    
    for coverage, height in layers:
        clouds = draw_layer(coverage, height)
        test_image = Image.alpha_composite(test_image, clouds)
    return test_image



interactive(children=(IntSlider(value=370, description='hour', max=741), Output()), _dom_classes=('widget-inte…